In [1]:
pip install transformers kobert-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
# 출처 : https://ai-creator.tistory.com/36
# 출처 : https://novice-engineers.tistory.com/9
# 출처 : https://github.com/uoneway/KoBertSum
import requests
from bs4 import BeautifulSoup
import json
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 뉴스 기사 URL 입력
url = input("뉴스 url을 입력해주세요: ")

# 웹 페이지 요청
response = requests.get(url)
if response.status_code == 200:
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 제목 크롤링
    title_tag = soup.find('h2', {'class': 'media_end_head_headline'})
    if title_tag:
        title = title_tag.get_text()
    else:
        title = "Title not found"

    # 기사 본문 크롤링
    content_tag = soup.find('div', {'id': 'newsct_article'})
    if content_tag:
        content = content_tag.get_text(strip=True)
    else:
        content = "Content not found"

    # KoBERT 모델과 토크나이저 로드
    model = BertModel.from_pretrained('monologg/kobert')
    tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

    # 텍스트 요약 함수
    def summarize_text(text, max_length=5):
        # 문장을 분리
        sentences = text.split('. ')
        # 문장을 토큰화하고 패딩 및 트렁케이션 적용
        inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            # 모델을 통해 문장 임베딩 생성
            outputs = model(**inputs)
        sentence_embeddings = outputs.last_hidden_state.mean(dim=1)

        # 문서 임베딩
        doc_embedding = sentence_embeddings.mean(dim=0).unsqueeze(0)
        # 유사도를 계산하여 상위 문장 선택
        similarities = cosine_similarity(doc_embedding, sentence_embeddings)
        top_indices = similarities.argsort()[0][-max_length:]

        # 선택된 문장을 요약으로 반환
        summary = [sentences[i] for i in top_indices]
        return ' '.join(summary)

    # 기사 요약
    summary = summarize_text(content, max_length=3)
    print("Summary:", summary)

    # 요약 내용을 로그로 출력하여 확인
    print("요약된 내용:", summary)

    # 카카오 API를 사용하여 요약 내용 전송
    def send_kakao_message(access_token, text):
        url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        data = {
            'template_object': json.dumps({
                'object_type': 'text',
                'text': text,
                'link': {
                    'web_url': 'https://developers.kakao.com',
                    'mobile_web_url': 'https://developers.kakao.com'
                }
            })
        }

        # 전송할 내용을 로그로 출력하여 확인
        print("전송할 내용:", text)

        response = requests.post(url, headers=headers, data=data)
        if response.status_code == 200:
            print("Message sent successfully")
        else:
            print(f"Failed to send message: {response.json()}")

    # 발급받은 액세스 토큰 사용
    access_token = '발급받은 액세스 토큰 입력'
    send_kakao_message(access_token, summary)  # 요약된 내용을 전송합니다.
else:
    print("Failed to retrieve the webpage")

뉴스 url을 입력해주세요: https://n.news.naver.com/article/005/0001705522?cds=news_media_pc&type=editn
Summary: 의·정 갈등이 여전하고 의사를 향한 비난 여론이 잠잠해지지 않자 의대 교수들이 그동안 챙기지 않았던 그들의 권리를 찾겠다고 주장하고 나선 것으로 풀이된다.한편 대한의사협회(의협)를 필두로 한 의료계는 무기한 휴진 방침을 철회하지 않고 있다 이 소송에서 법원은 ‘의대 교수는 사립학교법상 대학 교원으로 병원 근로자로서 지위는 인정되지 않는다’며 수당을 지급할 의무가 없다고 판결했다 김 회장은 이날 한 의료 전문지와 인터뷰에서 ‘의대 교수는 근로자가 아니다’라고 판단한 사법부 판결을 언급하며 “의대 교수들은 근로 계약서도 쓰지 않은 상태로 병원에서 일하고 있다
요약된 내용: 의·정 갈등이 여전하고 의사를 향한 비난 여론이 잠잠해지지 않자 의대 교수들이 그동안 챙기지 않았던 그들의 권리를 찾겠다고 주장하고 나선 것으로 풀이된다.한편 대한의사협회(의협)를 필두로 한 의료계는 무기한 휴진 방침을 철회하지 않고 있다 이 소송에서 법원은 ‘의대 교수는 사립학교법상 대학 교원으로 병원 근로자로서 지위는 인정되지 않는다’며 수당을 지급할 의무가 없다고 판결했다 김 회장은 이날 한 의료 전문지와 인터뷰에서 ‘의대 교수는 근로자가 아니다’라고 판단한 사법부 판결을 언급하며 “의대 교수들은 근로 계약서도 쓰지 않은 상태로 병원에서 일하고 있다
전송할 내용: 의·정 갈등이 여전하고 의사를 향한 비난 여론이 잠잠해지지 않자 의대 교수들이 그동안 챙기지 않았던 그들의 권리를 찾겠다고 주장하고 나선 것으로 풀이된다.한편 대한의사협회(의협)를 필두로 한 의료계는 무기한 휴진 방침을 철회하지 않고 있다 이 소송에서 법원은 ‘의대 교수는 사립학교법상 대학 교원으로 병원 근로자로서 지위는 인정되지 않는다’며 수당을 지급할 의무가 없다고 판결했다 김 회장은 이날 한 의료 전문지와 인터뷰에서 ‘의대 교수는 근로자가 아니다